## Split education column from the scrape

In [1]:
import pandas as pd
import numpy as np
import re

/Users/ilenapeng/.pyenv/versions/3.9.7/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Split name and school columns

In [2]:
df_raw = pd.read_csv('01-scrape.csv')
df_raw.head()

,name,school,circuit
0,"Colt, LeBaron Bradford","Yale College, A.B., 1868Columbia Law School, L...",First
1,"Putnam, William LeBaron","Bowdoin College, A.B., 1855Read law, 1858",First
2,"Lowell, Francis Cabot","Harvard University, A.B., 1876Harvard Law Scho...",First
3,"Schofield, William","Harvard Law School, LL.B., 1883",First
4,"Dodge, Frederic","Harvard College, B.A., 1867Harvard Law School,...",First


In [3]:
# testing splits with string
string_test = "St. John’s College, Annapolis, Maryland, A.B., 1913Harvard Law School, LL.B., 1916St. John’s College, Annapolis, Maryland, A.M., 1917"

print(re.split('(\d+)', string_test))

['St. John’s College, Annapolis, Maryland, A.B., ', '1913', 'Harvard Law School, LL.B., ', '1916', 'St. John’s College, Annapolis, Maryland, A.M., ', '1917', '']


In [4]:
# split school column 
education_df = df_raw["school"].str.split(r'(\d+)', expand=True)
education_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,"Yale College, A.B.,",1868,"Columbia Law School, LL.B.,",1870,,None,None,None,None,None,None
1,"Bowdoin College, A.B.,",1855,"Read law,",1858,,None,None,None,None,None,None
2,"Harvard University, A.B.,",1876,"Harvard Law School, LL.B.,",1879,,None,None,None,None,None,None
3,"Harvard Law School, LL.B.,",1883,,None,None,None,None,None,None,None,None
4,"Harvard College, B.A.,",1867,"Harvard Law School, LL.B.,",1869,,None,None,None,None,None,None


In [5]:
# Replace '' with None
education_df = education_df.replace('', None)

In [6]:
# Replace all Nones with NaNs
education_df = education_df.fillna(value=np.nan)

In [7]:
# drop columns that only have Nones 
# only column 10, tested using:
# test_df[10].unique()
# columns 8 & 9 each only have one entry

education_df = education_df.dropna(axis=1, how='all')
education_df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [8]:
# rename columns
column_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
new_names = ['school1', 'year1', 'school2', 'year2', 'school3', 'year3', 'school4', 'year4', 'school5', 'year5']
old_names = education_df.columns[column_indices]
education_df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [9]:
# merge dataframes
i = df_raw.columns.get_loc('school')

df = pd.concat([df_raw.iloc[:, :i], education_df, df_raw.iloc[:, i+1:]], axis=1)
df.head()

,name,school1,year1,school2,year2,school3,year3,school4,year4,school5,year5,circuit
0,"Colt, LeBaron Bradford","Yale College, A.B.,",1868,"Columbia Law School, LL.B.,",1870,,NaN,NaN,NaN,NaN,NaN,First
1,"Putnam, William LeBaron","Bowdoin College, A.B.,",1855,"Read law,",1858,,NaN,NaN,NaN,NaN,NaN,First
2,"Lowell, Francis Cabot","Harvard University, A.B.,",1876,"Harvard Law School, LL.B.,",1879,,NaN,NaN,NaN,NaN,NaN,First
3,"Schofield, William","Harvard Law School, LL.B.,",1883,"Harvard Law School, LL.B.,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,First
4,"Dodge, Frederic","Harvard College, B.A.,",1867,"Harvard Law School, LL.B.,",1869,NaN,NaN,NaN,NaN,NaN,NaN,First


In [10]:
# printed this to test that it worked
# df.school1.unique()

# some schools didn't split because they didn't have years, will manually fix these later

In [11]:
# split name columns
name_df = df["name"].str.split(',', expand=True)
name_df.head()

,0,1,2
0,Colt,LeBaron Bradford,None
1,Putnam,William LeBaron,None
2,Lowell,Francis Cabot,None
3,Schofield,William,None
4,Dodge,Frederic,None


In [12]:
# rename columns
column_indices = [0, 1, 2]
new_names = ['lastname', 'firstname', 'suffix']
old_names = name_df.columns[column_indices]
name_df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [13]:
# merge split names df and df with rest of information
df = pd.concat([name_df, df], axis=1)
df.head()

,lastname,firstname,suffix,name,school1,year1,school2,year2,school3,year3,school4,year4,school5,year5,circuit
0,Colt,LeBaron Bradford,None,"Colt, LeBaron Bradford","Yale College, A.B.,",1868,"Columbia Law School, LL.B.,",1870,,NaN,NaN,NaN,NaN,NaN,First
1,Putnam,William LeBaron,None,"Putnam, William LeBaron","Bowdoin College, A.B.,",1855,"Read law,",1858,,NaN,NaN,NaN,NaN,NaN,First
2,Lowell,Francis Cabot,None,"Lowell, Francis Cabot","Harvard University, A.B.,",1876,"Harvard Law School, LL.B.,",1879,,NaN,NaN,NaN,NaN,NaN,First
3,Schofield,William,None,"Schofield, William","Harvard Law School, LL.B.,",1883,"Harvard Law School, LL.B.,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,First
4,Dodge,Frederic,None,"Dodge, Frederic","Harvard College, B.A.,",1867,"Harvard Law School, LL.B.,",1869,NaN,NaN,NaN,NaN,NaN,NaN,First


In [14]:
# removing trailing spaces and commas
# test = 'Duquesne University, B.A., '
# line = test.strip().rstrip(',')
# print(line)

df = df.replace(r', $','', regex=True)
df.head()

,lastname,firstname,suffix,name,school1,year1,school2,year2,school3,year3,school4,year4,school5,year5,circuit
0,Colt,LeBaron Bradford,None,"Colt, LeBaron Bradford","Yale College, A.B.",1868,"Columbia Law School, LL.B.",1870,,NaN,NaN,NaN,NaN,NaN,First
1,Putnam,William LeBaron,None,"Putnam, William LeBaron","Bowdoin College, A.B.",1855,Read law,1858,,NaN,NaN,NaN,NaN,NaN,First
2,Lowell,Francis Cabot,None,"Lowell, Francis Cabot","Harvard University, A.B.",1876,"Harvard Law School, LL.B.",1879,,NaN,NaN,NaN,NaN,NaN,First
3,Schofield,William,None,"Schofield, William","Harvard Law School, LL.B.",1883,"Harvard Law School, LL.B.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,First
4,Dodge,Frederic,None,"Dodge, Frederic","Harvard College, B.A.",1867,"Harvard Law School, LL.B.",1869,NaN,NaN,NaN,NaN,NaN,NaN,First


In [15]:
# in school 1, there are some schools that appeared back to back, like this, because they had no years:
# University of PennsylvaniaBates College, A.B.
# further split the schools that had no years

# testing splits with string
string_test = "University of PennsylvaniaBates College, A.B."

string_new = re.sub(r'([a-z])([A-Z])', r'\1 | \2', string_test)
print(string_new)

print(string_new.split(' | '))

University of Pennsylvania | Bates College, A.B.
['University of Pennsylvania', 'Bates College, A.B.']


In [16]:
string_test = "University of PennsylvaniaBates College, A.B."

string_new = re.sub(r'([a-z])([A-Z])', r'\1|\2', string_test)
print(string_new)

print(string_new.split('|'))

University of Pennsylvania|Bates College, A.B.
['University of Pennsylvania', 'Bates College, A.B.']


In [17]:
# Add symbols in between lowercase so as to now break it when I later split
df = df.replace(".*DePau\w.*", "De###Paul University", regex=True)

In [18]:
df.school1 = df.school1.str.replace(r'([a-z])([A-Z])', r'\1|\2', regex=True)

In [19]:
split_df = df.school1.str.split('|', expand=True)
split_df.head(10)

,0,1,2
0,"Yale College, A.B.",None,None
1,"Bowdoin College, A.B.",None,None
2,"Harvard University, A.B.",None,None
3,"Harvard Law School, LL.B.",None,None
4,"Harvard College, B.A.",None,None
5,"Dartmouth College, A.B.",None,None
6,"Bowdoin College, A.B.",None,None
7,"Williams College, A.B.",None,None
8,University of Pennsylvania,"Bates College, A.B.",None
9,"Harvard College, A.B.",None,None


In [20]:
# Remove the ### I added in DePaul
split_df = split_df.replace("###", "", regex=True)

In [21]:
# rename columns
column_indices = [0, 1, 2]
new_names = ['school1_alt', 'school2_alt', 'school3_alt']
old_names = split_df.columns[column_indices]
split_df.rename(columns=dict(zip(old_names, new_names)), inplace=True)

In [22]:
# merge this new df and df with rest of information
df = pd.concat([split_df, df], axis=1)
df.head()

,school1_alt,school2_alt,school3_alt,lastname,firstname,suffix,name,school1,year1,school2,year2,school3,year3,school4,year4,school5,year5,circuit
0,"Yale College, A.B.",None,None,Colt,LeBaron Bradford,None,"Colt, LeBaron Bradford","Yale College, A.B.",1868,"Columbia Law School, LL.B.",1870,,NaN,NaN,NaN,NaN,NaN,First
1,"Bowdoin College, A.B.",None,None,Putnam,William LeBaron,None,"Putnam, William LeBaron","Bowdoin College, A.B.",1855,Read law,1858,,NaN,NaN,NaN,NaN,NaN,First
2,"Harvard University, A.B.",None,None,Lowell,Francis Cabot,None,"Lowell, Francis Cabot","Harvard University, A.B.",1876,"Harvard Law School, LL.B.",1879,,NaN,NaN,NaN,NaN,NaN,First
3,"Harvard Law School, LL.B.",None,None,Schofield,William,None,"Schofield, William","Harvard Law School, LL.B.",1883,"Harvard Law School, LL.B.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,First
4,"Harvard College, B.A.",None,None,Dodge,Frederic,None,"Dodge, Frederic","Harvard College, B.A.",1867,"Harvard Law School, LL.B.",1869,NaN,NaN,NaN,NaN,NaN,NaN,First


In [23]:
df.school1_alt.unique()

array(['Yale College, A.B.', 'Bowdoin College, A.B.',
       'Harvard University, A.B.', 'Harvard Law School, LL.B.',
       'Harvard College, B.A.', 'Dartmouth College, A.B.',
       'Williams College, A.B.', 'University of Pennsylvania',
       'Harvard College, A.B.',
       "St. John's College, Annapolis, Maryland, A.B.",
       'Brown University, A.B.', 'Columbia Law School',
       'Harvard University', 'College of the Holy Cross, A.B.',
       'Bates College, A.B.', 'Columbia University, B.A.',
       'Stanford University, A.B.',
       'University of Pennsylvania, Wharton School, B.S.',
       'College of the Holy Cross, B.S.', 'Harvard University, B.A.',
       'Tufts College, B.A.', 'Wellesley College, A.B.',
       'Haverford College, B.A.', 'Plymouth State College, B.A.',
       'Amherst College, B.A.', 'Brandeis University, B.A.',
       'Syracuse University', 'Columbia College, A.B.', 'Yale Law School',
       'Read law', 'University of Pennsylvania, B.A.',
       'Cornel

In [24]:
# delete the old school1 column, since these 3 new columns effectively take the place of it
del df['school1']

In [25]:
df.to_csv('02-split.csv', index=False)